In [19]:
# this file helps you run analysis on the most recent stock data
#DMI analysis 

# you need define what tickers and what time frame you want to sue for the analysis 

In [51]:
import pandas as pd
import datetime as dt
from datetime import date, timedelta
from datetime import datetime
import numpy as np
import holidays
import os
import ta
us_holidays = holidays.UnitedStates()

In [52]:
#Decalre all variables here: 
#your IEX tocken 
os.environ['TOKEN'] = "sk_b17067a1222f4aefb1f65b3d2643fce8"

#enter most recent date for which you downloaded trading data for
Global_todayDT =date(2020,7,18)
todayDT =Global_todayDT

# how many days/history of data would you like to use? 
N = 50

# define local path where the data is saved: 
location  = "/Users/yaninashevchenko/Desktop/IEX_historical-prices/script/output"

#DMI period 
#DMI calculataion on a 1 day bar
lens = 14
ADX_P = 20

In [53]:
#data analysis and functions
#it should be a working day (Monday-Friday) 

def lasttradingDT(date):
    if (date.weekday() < 5 and not date in us_holidays ): XX = date 
    elif (date.weekday() == 0 and date in us_holidays ): XX = date - dt.timedelta(days=3)
    elif (date.weekday() < 5 and date in us_holidays ): XX = date - dt.timedelta(days=1)     
    elif date.weekday() == 5: XX = date - dt.timedelta(days=1)
    elif date.weekday() == 6: XX = date - dt.timedelta(days=2)
    return XX
lastDT = lasttradingDT(todayDT) 

#add 0 to month notation 
def Month_udf(date):
    if date.month<10: monthDT = "0" +str(date.month)
    else: monthDT =date.month
    return monthDT
monthDT  = Month_udf(lastDT)

# add 0 to day notation 
def DA(date):
    if date.day<10: 
        dayDT = "0" +str(date.day)
    else: dayDT =date.day
    return dayDT

dayDT  = DA(lastDT)

#actual week order 
AW = lastDT.isocalendar()[1]

In [54]:
# this part helps define tickers you want to get data for: 

#get SP500 tickers 
import bs4 as bs
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

SP500 = save_sp500_tickers()
j=0
mylst=SP500
for i in SP500:
    SP500[j]=i.rstrip("\n")
    j+=1
#----------------------------------
# or select a few tickers manually 
Ticker_test = ['SPY', 'AAPL', 'BABA']

#----------------------------------
#all trading tickers ~8800
def get_tickers():
    
    base_url = 'https://cloud.iexapis.com/stable/ref-data/iex/symbols?format=csv&token={}'
    df = pd.read_csv(base_url.format(os.environ['TOKEN']))
    #df.to_csv('{}/tickers_all.csv'.format(path))
    tickers = df[~df['isEnabled'] == False].loc[:, 'symbol'].tolist()

    return(tickers)
Full_list  = get_tickers()

#----------------------------------
#----------------------------------
#----------------------------------

#indicate what ticker list you want to use: Ticker_test or SP500 or Full_list? 
ticker_list  = SP500

In [55]:
# this part pulls data for N days and ticker list you selected (SP500 or Ticker_test)

In [56]:
frames = {}
x=0

for ii in ticker_list: 
    todayDT =Global_todayDT
    Ticker = ii

    #lets read latest N days data for each Ticker
    append_data = pd.DataFrame()

    variables = locals()
    for i in range(N):
        if i==0: DT = todayDT
        else: DT = todayDT - dt.timedelta(days=1)
        lastDT = lasttradingDT(DT)
        monthDT  = Month_udf(lastDT)
        dayDT  = DA(lastDT)
        AW = lastDT.isocalendar()[1]

        test0 = os.path.isfile("{}/{}/{}-CW{}/{}{}{}/DONE/{}{}{}_{}.csv".format(location, lastDT.year, lastDT.year, AW, lastDT.year, monthDT, dayDT, lastDT.year, monthDT, dayDT, Ticker))
        x=0
        if test0 == True: 
            test = pd.read_csv("{}/{}/{}-CW{}/{}{}{}/DONE/{}{}{}_{}.csv".format(location, lastDT.year, lastDT.year, AW, lastDT.year, monthDT, dayDT, lastDT.year, monthDT, dayDT, Ticker))
            append_data= append_data.append(test)
            variables["{0}".format(Ticker)] = append_data
            x=1
        todayDT = lastDT
        print(ii, i, todayDT )
        
    # add dataframe to a dict 
    if x == 1: 
        frames[ii]=variables["{0}".format(Ticker)]

MMM 0 2020-07-17
MMM 1 2020-07-16
MMM 2 2020-07-15
MMM 3 2020-07-14
MMM 4 2020-07-13
MMM 5 2020-07-10
MMM 6 2020-07-09
MMM 7 2020-07-08
MMM 8 2020-07-07
MMM 9 2020-07-06
MMM 10 2020-07-03
MMM 11 2020-07-02
MMM 12 2020-07-01
MMM 13 2020-06-30
MMM 14 2020-06-29
MMM 15 2020-06-26
MMM 16 2020-06-25
MMM 17 2020-06-24
MMM 18 2020-06-23
MMM 19 2020-06-22
MMM 20 2020-06-19
MMM 21 2020-06-18
MMM 22 2020-06-17
MMM 23 2020-06-16
MMM 24 2020-06-15
MMM 25 2020-06-12
MMM 26 2020-06-11
MMM 27 2020-06-10
MMM 28 2020-06-09
MMM 29 2020-06-08
MMM 30 2020-06-05
MMM 31 2020-06-04
MMM 32 2020-06-03
MMM 33 2020-06-02
MMM 34 2020-06-01
MMM 35 2020-05-29
MMM 36 2020-05-28
MMM 37 2020-05-27
MMM 38 2020-05-26
MMM 39 2020-05-22
MMM 40 2020-05-21
MMM 41 2020-05-20
MMM 42 2020-05-19
MMM 43 2020-05-18
MMM 44 2020-05-15
MMM 45 2020-05-14
MMM 46 2020-05-13
MMM 47 2020-05-12
MMM 48 2020-05-11
MMM 49 2020-05-08
ABT 0 2020-07-17
ABT 1 2020-07-16
ABT 2 2020-07-15
ABT 3 2020-07-14
ABT 4 2020-07-13
ABT 5 2020-07-10
ABT 6 20

In [57]:
#company=row['Company Name'] # Get your company name as a string from your summary.
#frames['BABA'] # Returns a DataFrame.
len(frames)


505

In [25]:
#now all ticker data was uploaded in momery under the ticker name as part of frames dict 

In [58]:
#Lets loop through dataframes and calculate DMI flags for each ticker  

dfPipe =  pd.DataFrame()
proxy_GlobalDT =  Global_todayDT
todayDT = proxy_GlobalDT

DT = lasttradingDT(todayDT)
pipe = pd.DataFrame()

for ii in ticker_list: 
# consolidate time features into time stamps 
    df =  frames[ii]
    todayDT = proxy_GlobalDT

    df['year'] = df.date.astype(str).str[:4]
    df['month'] = df.date.astype(str).str[6:7]
    df['day'] = df.date.astype(str).str[8:10]
    df['hour'] = df.minute.astype(str).str[:2]
    df['min'] = df.minute.astype(str).str[3:5]
    df['min1'] = df['min'].astype(int)
    df["stamp"] = pd.to_datetime(df[['year', 'month', 'day', 'hour']]) + pd.to_timedelta(df['min1'], unit='m')
    # order by date 
    df1 = df.sort_values("stamp")
    #add today's aggregates 
    df1['max_day'] =df1.groupby('date')['close'].transform('max')
    df1['min_day'] =df1.groupby('date')['close'].transform('min')

    #end of the day close price points 
    l0 = df.sort_values('stamp').groupby('date').nth(-1)
    l2=l0.rename({"close": "close_day"}, axis = 1)
    l3 = l2[['close_day']]
    l4 = l3[['close_day']].rename({"close_day": "close_day_prev"}, axis = 1)
    l5 = pd.concat([l3, l4.shift()], axis=1)

    #join the prev close dates as new cloumns
    df11 = df1.merge(l5, left_on = 'date', right_index = True).reset_index()
    #group by dates to start working at the day level (not minute)
    df3 = df11.groupby('date').last()
    # calcualte previous day min and max 
    rr =df3[['max_day','min_day']].rename({"max_day": "max_day_prev", "min_day": "min_day_prev"}, axis = 1)
    df3 = pd.concat([df3, rr.shift()], axis=1)
    df3['date'] = df3.index
    df3["date1"] =pd.to_datetime(df3['date'])
    df9 = None
    df9 = df3.dropna(subset=['date', 'close'], how='all')#,axis=0
   
    RSI_P = 15
    k = ta.trend.ADXIndicator(df9['max_day'], df9['min_day'], df9['close'], lens, True)
    #RSI = ta.momentum.RSIIndicator(df9['close'], RSI_P)
    
    df9 ['ADX14'] = k.adx()
    df9 ['ADX_neg'] = k.adx_neg()
    df9 ['ADX_pos'] = k.adx_pos()
    #lets offset the DMI data by 1 and 2 days 
    rr =df9[['ADX_pos','ADX_neg']].rename({"ADX_pos": "ADX_pos_prev", "ADX_neg": "ADX_neg_prev"}, axis = 1)
    rr2 =df9[['ADX_pos','ADX_neg']].rename({"ADX_pos": "ADX_pos_prev2", "ADX_neg": "ADX_neg_prev2"}, axis = 1)
    df9_2 = pd.concat([df9, rr.shift(1)], axis=1)
    df10 = pd.concat([df9_2, rr2.shift(2)], axis=1)

    

    #Calculate DMI crossing (if there is any)
    def crossDMI_bull(row):
        if row["ADX_pos"] > row["ADX_neg"] and row["ADX_pos_prev"] <= row["ADX_neg_prev"]:
            return 1
        else:
            return 0
    
    def crossDMI_bull2d(row):
        if row["ADX_pos"] > row["ADX_neg"] and row["ADX_pos_prev2"] <= row["ADX_neg_prev2"]:
            return 1
        else:
            return 0
        
    def crossDMI_bear(row):
        if row["ADX_pos"] < row["ADX_neg"] and row["ADX_pos_prev"] >= row["ADX_neg_prev"]:
            return 1
        else:
            return 0
        
    def crossDMI_bear2d(row):
        if row["ADX_pos"] < row["ADX_neg"] and row["ADX_pos_prev2"] >= row["ADX_neg_prev2"]:
            return 1
        else:
            return 0
        
    df10 = df10.assign(DMI_cross_bear=df10.apply(crossDMI_bear, axis=1))
    df10 = df10.assign(DMI_cross_bull=df10.apply(crossDMI_bull, axis=1))
    df10 = df10.assign(DMI_cross_bull_2d=df10.apply(crossDMI_bull2d, axis=1))
    df10 = df10.assign(DMI_cross_bear_2d=df10.apply(crossDMI_bear2d, axis=1))
    
    df11 =df10
    df12 = dfPipe.append(df11[['symbol', 'date1', 'DMI_cross_bear','DMI_cross_bull', 'DMI_cross_bear_2d','DMI_cross_bull_2d']][df11['date1'].dt.date == DT])
    pipe= pipe.append(df12)
   
    del df3
    del k

/Users/yaninashevchenko/anaconda3/lib/python3.6/site-packages/ta/trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/Users/yaninashevchenko/anaconda3/lib/python3.6/site-packages/ta/trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [59]:
pipe

,symbol,date1,DMI_cross_bear,DMI_cross_bull,DMI_cross_bear_2d,DMI_cross_bull_2d
date,,,,,,
2020-07-17,MMM,2020-07-17,0,0,0,0
2020-07-17,ABT,2020-07-17,0,0,0,0
2020-07-17,ABBV,2020-07-17,0,0,0,0
2020-07-17,ABMD,2020-07-17,0,0,0,0
2020-07-17,ACN,2020-07-17,0,0,0,0
2020-07-17,ATVI,2020-07-17,0,0,0,0
2020-07-17,ADBE,2020-07-17,0,0,0,0
2020-07-17,AMD,2020-07-17,0,0,0,0
2020-07-17,AAP,2020-07-17,0,0,0,0


In [70]:
#AA = pipe.loc[pipe.DMI_cross_bear >0]
AA = pipe[(pipe.DMI_cross_bear >0) | (pipe.DMI_cross_bull >0) | (pipe.DMI_cross_bull_2d >0) | (pipe.DMI_cross_bear_2d >0)]
AA1 = pipe[(pipe.DMI_cross_bear >0) | (pipe.DMI_cross_bull >0)]
AA2 = pipe[pipe.DMI_cross_bull >0]

In [71]:
AA2.count()

symbol               21
date1                21
DMI_cross_bear       21
DMI_cross_bull       21
DMI_cross_bear_2d    21
DMI_cross_bull_2d    21
dtype: int64

In [72]:
AA2

,symbol,date1,DMI_cross_bear,DMI_cross_bull,DMI_cross_bear_2d,DMI_cross_bull_2d
date,,,,,,
2020-07-17,ANET,2020-07-17,0,1,0,0
2020-07-17,ADSK,2020-07-17,0,1,0,0
2020-07-17,BLL,2020-07-17,0,1,0,1
2020-07-17,BAX,2020-07-17,0,1,0,1
2020-07-17,COG,2020-07-17,0,1,0,1
2020-07-17,CSCO,2020-07-17,0,1,0,0
2020-07-17,CTXS,2020-07-17,0,1,0,0
2020-07-17,CCI,2020-07-17,0,1,0,0
2020-07-17,CVS,2020-07-17,0,1,0,1
